## 路径还原-模糊匹配

In [1]:
include("../src/CCKS-mKGQA.jl")
include("../src/tools/regexpr.jl") # 正则表达
include("../src/loaddata/extractdata.jl") # 原始数据
include("../src/loaddata/translatedata.jl") # 翻译数据
include("../src/tools/ettalign.jl") # 对齐工具
include("../src/tools/distance.jl") # 距离函数
include("../src/process/submitpath.jl") # 数据提交
include("../src/datatype.jl")
mkpath("submit")

"submit"

In [78]:
illpath = "ILLs_official"
include("../src/loaddata/illsdata.jl");

### 生成结果

In [29]:
tmp = precise_submit("Estonian_election_election,_2003", ["after_election", "party"], illsdata)
# println.(triple_info.(partition(split(tmp, '#'), 3)))

2-element Vector{String}:
 "<http://dbpedia.org/resource/Es" ⋯ 221 bytes ⋯ "dbpedia.org/resource/爱沙尼亚共和国党>"
 "<http://dbpedia.org/resource/Es" ⋯ 221 bytes ⋯ "dbpedia.org/resource/祖國與共和國聯盟>"

In [92]:
illpath = "ILLs_official"
include("../src/loaddata/illsdata.jl")
println(illsdata, '\t', length(illsdata.mtILLs))
for i in 1:8
    illpath = "ILLs_$i"
    include("../src/loaddata/illsdata.jl")
    println(illsdata, '\t', length(illsdata.mtILLs))
    flush(stdout)
end
illpath = "ILLs_wiki"
include("../src/loaddata/illsdata.jl")
println(illsdata, '\t', length(illsdata.mtILLs))

"ILLs_official"	9802
"ILLs_1"	23998
"ILLs_2"	18441
"ILLs_3"	37119
"ILLs_4"	62259
"ILLs_5"	13132
"ILLs_6"	19828
"ILLs_7"	52388
"ILLs_8"	50565
"ILLs_wiki"	29655


In [60]:
submit_id = "deduct-ills4"
sols = split.(strip.(split(strip(read(open("predict/predict_data_bertsim.txt", "r"), String)), '\n')),'\t')
noans = "<http://dbpedia.org/resource/Francis_Russell,_Marquess_of_Tavistock>#<http://dbpedia.org/property/parents>#<http://dbpedia.org/resource/John_Russell,_4th_Duke_of_Bedford>#<http://zh.dbpedia.org/resource/第四代贝德福德公爵约翰·罗素>#<http://zh.dbpedia.org/property/successor>#<http://zh.dbpedia.org/resource/丹尼尔·芬奇，第八代温奇尔西伯爵>";
# 返回结果
res = Vector{String}(undef, 1500)
fails, notgood = Int[], Int[]
## 精确匹配
for (i, sol) in enumerate(sols)
    que, rels = ill_valid_ques[i], sol[3:end]
    ner, rawner = ill_valid_ners[i], valid_ners[i]
    # 精确路径
    submits = precise_submit(ner, rels, illsdata)
    isempty(submits) && (push!(fails, i);continue) # 无解
    bettersubmits = filter(contains(rawner), submits) # 不需要实体链接
    isempty(bettersubmits) ? push!(notgood, i) : (submits = bettersubmits)
    res[i] = first(submits)
end
println(length(fails), '\t', length(notgood))
flush(stdout)
res[fails] .= noans
yestoday = copy(res)
## 模糊匹配
vaguefails, vague_inds, vague_res, vaguenotgood = Int[], Int[], String[], Int[]
for (i, sol) in enumerate(sols)
    (i ∉ fails || length(sol) ≤ 3) && continue
    que, rels = ill_valid_ques[i], sol[3:end]
    ner, rawner = ill_valid_ners[i], valid_ners[i]
    # 精确路径
    submits = vague_submit(ner, rels, illsdata)
    isempty(submits) && (push!(vaguefails, i);continue) # 无解
    bettersubmits = filter(contains(rawner), submits) # 不需要实体链接
    isempty(bettersubmits) ? push!(vaguenotgood, i) : (submits = bettersubmits)
    push!(vague_res, first(submits))
    push!(vague_inds, i)
    res[i] = first(submits)
end
println(length(fails) - length(vague_inds), '\t', length(vaguenotgood))

286	15
182	7


In [42]:
res[427] = res[526] = res[317]
res[986] = res[688] = precise_submit("Ride_Me_(album)", ["title", "lyrics", "profession"], illsdata)[1]
res[1194] = precise_submit("Ride_Me_(album)", ["title", "lyrics", "education_level"], illsdata)[1]
today = res

1500-element Vector{String}:
 "<http://dbpedia.org/resource/Cé" ⋯ 199 bytes ⋯ ".dbpedia.org/resource/阿希尔·戈尔基>"
 "<http://dbpedia.org/resource/20" ⋯ 406 bytes ⋯ "esource/Duchesne_County,_Utah>"
 "<http://dbpedia.org/resource/He" ⋯ 314 bytes ⋯ "://zh.dbpedia.org/resource/歌手>"
 "<http://dbpedia.org/resource/UE" ⋯ 349 bytes ⋯ "://zh.dbpedia.org/resource/巴黎>"
 "<http://dbpedia.org/resource/Ta" ⋯ 214 bytes ⋯ "//zh.dbpedia.org/resource/梁肅戎>"
 "<http://dbpedia.org/resource/Un" ⋯ 246 bytes ⋯ "zh.dbpedia.org/resource/迪克·切尼>"
 "<http://dbpedia.org/resource/Ho" ⋯ 204 bytes ⋯ "://zh.dbpedia.org/resource/歌手>"
 "<http://dbpedia.org/resource/Fr" ⋯ 216 bytes ⋯ "h.dbpedia.org/resource/欧根·斯普勒>"
 "<http://dbpedia.org/resource/20" ⋯ 377 bytes ⋯ "rg/resource/Zosimades_Stadium>"
 "<http://dbpedia.org/resource/Ci" ⋯ 243 bytes ⋯ ".dbpedia.org/resource/捷克共和国总统>"
 "<http://dbpedia.org/resource/El" ⋯ 253 bytes ⋯ "bpedia.org/resource/香港立法會議員列表>"
 "<http://dbpedia.org/resource/Es" ⋯ 221 bytes ⋯ "dbpedia.org/resourc

In [43]:
submit_io = open("submit/submit_$submit_id.txt", "w")
demo_io = open("submit_test/demo_$submit_id.txt", "w")
println(submit_io, "id\tans_path")
for (i, sol) in enumerate(res)
    println(submit_io, i-1, '\t', sol)
    info = triple_info.(partition(split(sol, '#'), 3))
    sol == noans && continue
    println(demo_io, i-1, '\t', join(mt_valid_ques_ner[i], '\t'))
    println.(Ref(demo_io), info)
    println(demo_io)
end
close(submit_io)
close(demo_io)

In [41]:
for ind in findall(i->i[2]=="Ride_Me_(album)", valid_ques_ner)
    println(ind, '|', mt_valid_ques[ind], '|', String.(sols[ind][3:end]))
end

317|what is the language of the lyricist of the title of NER|["title", "lyrics", "language"]
427|do You Know what Language Lyrics Of NER's Title Is Using|["title", "lyrics"]
526|do You Know what Language Lyrics Of NER's Title Is Using|["title", "lyrics"]
688|what is the job of the lyrics of the title of NER?|["title", "lyrics"]
986|what does a lyricist with the title of NER do?|["title", "lyrics"]
1194|where is the lyrics of the title of NER educated?|["title", "lyrics"]


### 数据分析

In [44]:
# 数据比对
cmp = open("submit_test/compare_yes_tod.txt", "w")
i = j = 0
name1, name2 = "Before", "After"
for (k, que, ner, res1, res2) in zip(1:1500, mt_valid_ques, valid_ners, yestoday, today)
    res1 == res2 && continue
    println(cmp, k, '\t', que, '\t', ner)
    if res1 == noans
        println(cmp, "$name1\nNONE\n$name2")
        println.(Ref(cmp), triple_info.(partition(split(res2, '#'), 3)))
        j += 1
    elseif res2 == noans
        println(cmp, "$name1")
        println.(Ref(cmp), triple_info.(partition(split(res1, '#'), 3)))
        println(cmp, "$name2\nNONE")
    else
        println(cmp, "$name1")
        println.(Ref(cmp), triple_info.(partition(split(res1, '#'), 3)))
        println(cmp, "$name2")
        println.(Ref(cmp), triple_info.(partition(split(res2, '#'), 3)))
    end
    println(cmp)
    i += 1
end
close(cmp)
println("数据差异 $i, $name2 新解数 $j")

数据差异 34, After 新解数 34
